In [1]:
import tensorflow as tf
import numpy as np

C:\Users\t\Anaconda3\envs\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
Nsample = 1000

In [6]:
Nfactor = 10

In [7]:
trX = np.random.randn(Nsample , Nfactor)

In [9]:
trX.shape

(1000, 10)

In [11]:
Nregime  = 2

In [13]:
A = np.random.randn(Nfactor , Nregime)

In [2]:
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33 # create a y value which is approximately linear but with some random noise

In [17]:
trZ = np.dot(trX , A)

In [18]:
trZ.shape

(1000, 2)

In [19]:
trZ[1].shape

(2,)

In [20]:
trX2 = np.random.randn(Nsample, 1)

In [21]:
trX2.shape

(1000, 1)

In [24]:
trY = [z[0] if  trX2[i , 0] > 0 else z[1] for i , z in enumerate(trZ)]